In [1]:
import pandas as pd
import numpy as np
import math
from scipy.spatial import cKDTree
from scipy.optimize import curve_fit
import warnings

warnings.filterwarnings("ignore")

In [2]:
main = pd.read_csv("data/total.csv")
water = pd.read_csv("data/water_velocity.csv")
wind = pd.read_csv("data/wind_velocity.csv")

In [3]:
water = water.fillna(0)
wind = wind.fillna(0)

In [4]:
main["water_u"] = 0
main["water_v"] = 0
main["wind_u"] = 0
main["wind_v"] = 0
main['d_long'] = main['longitude'].diff().fillna(0)
main['d_lati'] = main['latitude'].diff().fillna(0)

main = main.groupby('drifter').apply(lambda group: group.iloc[1:]).reset_index(drop=True)

In [5]:
# Create a grid of longitude and latitude from the water DataFrame
grid = water[['longitude', 'latitude']].drop_duplicates().values

# Create a KD-Tree for efficient nearest neighbor search
kdtree = cKDTree(grid)

# Function to find the 4 closest vertices for a given point
def find_closest_vertices(row):
    point = np.array([row['longitude'], row['latitude']])
    # Query the KD-Tree to find the 4 closest vertices
    _, indices = kdtree.query(point, k=4)
    return indices

# Apply the function to the main DataFrame to find the 4 closest vertices for each point
main['closest_vertices'] = main.apply(find_closest_vertices, axis=1)

In [6]:
r = []
theta = []
for i in range(len(main)):
    x = water.loc[water["time"] == main.iloc[i][1]]
    w_d = []
    w_r = []
    w_theta = []
    
    for j in main.iloc[i][-1]:
        d = ((main.iloc[i][2] - x.iloc[j][1])**2 + (main.iloc[i][3] - x.iloc[j][2])**2) ** 0.5
        _r = ((x.iloc[j][-2] ** 2) + (x.iloc[j][-1] ** 2)) * 0.5
        
        u = np.array([x.iloc[j][-2], 0])
        v = np.array([0, x.iloc[j][-1]])
        dot_product = np.dot(u, u + v)
        u_norm = np.linalg.norm(u)
        u_plus_v_norm = np.linalg.norm(u+v)
        
        cos_theta = dot_product / (u_norm * u_plus_v_norm)
        _theta = math.acos(cos_theta)
        
        w_d.append(d)
        w_r.append(_r)
        w_theta.append(_theta)
        
    total_distance = sum(w_d)
    weights = [1 - (distances / total_distance) for distances in w_d]

    # Perform quadratic interpolation
    weighted_r = sum(r * weight for r, weight in zip(w_r, weights))
    weighted_theta = sum(theta * weight for theta, weight in zip(w_theta, weights))
    
    r.append(weighted_r)
    theta.append(weighted_theta)

In [7]:
w_u = []
w_v = []

for i in range(len(r)):
    u = r[i] * math.cos(theta[i])
    v = r[i] * math.sin(theta[i])
    w_u.append(u)
    w_v.append(v)
    
main["water_u"] = w_u
main["water_v"] = w_v

In [8]:
# Create a grid of longitude and latitude from the water DataFrame
grid2 = wind[['longitude', 'latitude']].drop_duplicates().values

# Create a KD-Tree for efficient nearest neighbor search
kdtree = cKDTree(grid2)

# Function to find the 4 closest vertices for a given point
def find_closest_vertices(row):
    point = np.array([row['longitude'], row['latitude']])
    # Query the KD-Tree to find the 4 closest vertices
    _, indices = kdtree.query(point, k=4)
    return indices

# Apply the function to the main DataFrame to find the 4 closest vertices for each point
main['closest_vertices2'] = main.apply(find_closest_vertices, axis=1)

In [9]:
r = []
theta = []
for i in range(len(main)):
    x = wind.loc[wind["time"] == main.iloc[i][1]]
    w_d = []
    w_r = []
    w_theta = []
    
    for j in main.iloc[i][-1]:
        d = ((main.iloc[i][2] - x.iloc[j][1])**2 + (main.iloc[i][3] - x.iloc[j][2])**2) ** 0.5
        _r = ((x.iloc[j][-2] ** 2) + (x.iloc[j][-1] ** 2)) * 0.5
        
        u = np.array([x.iloc[j][-2], 0])
        v = np.array([0, x.iloc[j][-1]])
        dot_product = np.dot(u, u + v)
        u_norm = np.linalg.norm(u)
        u_plus_v_norm = np.linalg.norm(u+v)
        
        cos_theta = dot_product / (u_norm * u_plus_v_norm)
        _theta = math.acos(cos_theta)
        
        w_d.append(d)
        w_r.append(_r)
        w_theta.append(_theta)
        
    total_distance = sum(w_d)
    weights = [1 - (distances / total_distance) for distances in w_d]

    # Perform quadratic interpolation
    weighted_r = sum(r * weight for r, weight in zip(w_r, weights))
    weighted_theta = sum(theta * weight for theta, weight in zip(w_theta, weights))
    
    r.append(weighted_r)
    theta.append(weighted_theta)

In [10]:
w_u = []
w_v = []

for i in range(len(r)):
    u = r[i] * math.cos(theta[i])
    v = r[i] * math.sin(theta[i])
    w_u.append(u)
    w_v.append(v)
    
main["wind_u"] = w_u
main["wind_v"] = w_v

In [11]:
for i in range(100):
    print(theta[i])

0.9688992139418305
1.1435814209932285
0.7595108739995341
3.017604120048401
1.5178167282960082
0.10059312140013102
0.9568726376668975
1.271449705023084
1.247438690920133
2.941497493060543
2.369413735120533
2.7219455234198024
4.545625019354112
1.7556068280545774
3.3355136515635255
2.850188307385196
3.191401942632992
3.1054616331236753
3.583476500111103
2.8362968827378143
3.58680260253021
2.8433148395569514
3.3682762966562243
0.33837697914064124
0.711673724871021
3.236648365819276
4.130676837858218
4.314870581061463
3.0882020419290033
3.610845296915929
3.141103803249428
3.2571225616724027
0.3850381272698787
3.1033350203039913
1.0903141536979652
0.10149433880279227
2.5451036962782063
1.8311640605449433
3.083804988150216
4.551912011955629
4.419852685824087
3.735758474106572
2.3353063755940497
1.5716134766787913
2.1412564872452053
2.9750768405768038
3.1193469718993647
1.1901253171994945
1.7018649094747895
3.9634582997638863
4.384131097063417
4.552803460027008
1.1776243736765224
0.76631156640

In [12]:
main

,drifter,time,longitude,latitude,water_u,water_v,wind_u,wind_v,d_long,d_lati,closest_vertices,closest_vertices2
0,0,51.50,129.401993,35.020000,-0.019187,0.028549,15.826964,23.040293,0.090988,0.070000,"[1862, 1770, 1863, 1771]","[6830, 6738, 6831, 6739]"
1,0,51.75,129.514008,35.061001,-0.021663,0.025037,11.671312,25.636899,0.112015,0.041000,"[1863, 1862, 1771, 1770]","[6739, 6831, 6738, 6830]"
2,0,52.00,129.524002,35.032001,-0.035334,0.046780,2.051413,1.947859,0.009995,-0.028999,"[1863, 1862, 1771, 1770]","[6831, 6739, 6830, 6738]"
3,0,52.25,129.626007,35.083000,0.004708,0.163147,-164.508211,20.502301,0.102005,0.050999,"[1863, 1771, 1864, 1862]","[6739, 6831, 6740, 6738]"
4,0,52.50,129.712997,35.058998,0.189563,0.127300,18.975803,357.836720,0.086990,-0.024002,"[1863, 1864, 1771, 1772]","[6739, 6831, 6740, 6832]"
...,...,...,...,...,...,...,...,...,...,...,...,...
15809,30,638.00,147.348999,44.167999,0.004412,0.010521,8.612322,14.220886,0.029999,-0.000999,"[5243, 5335, 5151, 5427]","[3495, 3587, 3403, 3679]"
15810,30,638.25,147.307007,44.138000,0.003963,0.010368,-9.932827,3.462747,-0.041992,-0.029999,"[5243, 5335, 5151, 5427]","[3495, 3587, 3403, 3679]"
15811,30,638.50,147.250000,44.146999,0.008781,0.006901,-23.999684,-19.684842,-0.057007,0.008999,"[5243, 5335, 5151, 5427]","[3495, 3587, 3403, 3679]"
15812,30,638.75,147.322998,44.201000,0.008143,0.007125,-11.668403,-34.492430,0.072998,0.054001,"[5243, 5335, 5151, 5427]","[3495, 3403, 3587, 3311]"


In [13]:
main.to_csv("data/total_add_velocity_bilinear_interpolation_polar_coordinate.csv", index = False)

In [14]:
main.describe()

,drifter,time,longitude,latitude,water_u,water_v,wind_u,wind_v,d_long,d_lati
count,15814.000000,15814.000000,15814.000000,15814.000000,14038.000000,14038.000000,15814.000000,15814.000000,15814.000000,15814.000000
mean,15.866384,341.630644,134.255299,38.418786,-0.024254,0.021742,-21.693606,23.052652,0.014679,0.006684
std,8.613671,140.691731,3.622366,2.460060,0.081185,0.079319,72.271495,81.469318,0.053495,0.043642
min,0.000000,51.500000,125.299004,33.140999,-0.821871,-0.666562,-599.974976,-575.695675,-0.230011,-0.247002
25%,9.000000,240.250000,131.248001,36.632000,-0.049608,-0.003427,-49.045611,-8.337270,-0.016998,-0.016998
50%,16.000000,348.000000,133.824501,37.919998,-0.015298,0.014182,-13.119893,12.604744,0.010010,0.006001
75%,23.000000,436.250000,136.547752,39.633999,0.002581,0.046321,6.751716,52.611333,0.043991,0.031002
max,30.000000,666.000000,148.697998,46.507000,1.495287,1.895010,507.175410,572.008624,0.475006,0.247002
